# Proposed Ensemble Models

Given the constraints and objectives, I recommend considering the following models for the ensemble:
	
    1.	Model 1: LSTM Network on Raw GPS Data
    
>•	Input Data: Sequences of raw GPS data (speed, progress, stride_frequency, etc.).

>•	Architecture: An LSTM network designed to capture temporal dependencies and patterns in the sequential data.

>•	Advantage: LSTMs are well-suited for time-series data and can learn complex temporal dynamics without the need for hand-engineered features like acceleration.

    2.	Model 2: 1D Convolutional Neural Network (1D-CNN)
	
>•	Input Data: The same raw GPS sequences as in Model 1.

>•	Architecture: A 1D-CNN that applies convolutional filters across the time dimension to detect local patterns.

>•	Advantage: CNNs can capture spatial hierarchies and are effective in recognizing patterns in sequences, potentially identifying features like sudden changes in speed or stride frequency.

    3.	Model 3: Transformer-based Model
	
>•	Input Data: Raw GPS sequences and possibly sectionals data.

>•	Architecture: A Transformer model that uses self-attention mechanisms to weigh the importance of different parts of the sequence.

>•	Advantage: Transformers can model long-range dependencies and focus on the most relevant parts of the sequence for prediction.

## Additional Models (Optional):

    4.	Model 4: Gated Recurrent Unit (GRU) Network

>•	Similar to LSTMs but with a simpler architecture, GRUs can be more efficient and may perform better on certain datasets.

>•	Model 5: Temporal Convolutional Network (TCN)

>•	TCNs are designed for sequential data and can capture long-term dependencies using causal convolutions and residual connections.


# The LSTM Network on Raw GPS Data

Initially I desired to merge the GPS data with Sectionals, but the timestamp and gate_name intervals of each respectively made it difficult to align the data in sequences -- something that is needed for Long-Short Term Memory models. Therefore, it was decided to go with an ensemble approach. There will be additional models that incorporate Equibase data as well, but for the time being, the focus will be on Total Performance GPS data. 

In [2]:
# Environment setup

import logging
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import geopandas as gpd
from datetime import datetime
import configparser
from src.data_ingestion.ingestion_utils import (
    get_db_connection, update_tracking, load_processed_files
)
from src.data_ingestion.eqb_ppData import process_pluspro_data
from src.data_ingestion.eqb_resultsCharts import process_resultscharts_data
from src.data_ingestion.tpd_datasets import (
    process_tpd_sectionals_data,
    process_tpd_gpsdata_data
)
import traceback

# Load the configuration file
config = configparser.ConfigParser()
config.read('/home/exx/myCode/horse-racing/FoxRiverAIRacing/config.ini')

# Set up logging for consistent logging behavior in Notebook
logging.basicConfig(level=logging.INFO)

# Retrieve database credentials from config file
# Retrieve database credentials from config file
db_host = config['database']['host']
db_port = config['database']['port']
db_name = config['database']['dbname']  # Corrected from 'name' to 'dbname'
db_user = config['database']['user']

# Establish connection using get_db_connection
conn = get_db_connection(config)

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_user}@{db_host}:{db_port}/{db_name}')

In [3]:
query_results = """
   SELECT course_cd, race_date, race_number, program_num, official_fin, post_pos
    FROM v_results_entries
    WHERE breed = 'TB';
"""

query_sectionals = """    
    
SELECT course_cd, race_date, race_number, saddle_cloth_number, gate_numeric, length_to_finish, 
    sectional_time, running_time, distance_back, distance_ran, number_of_strides 
FROM v_sectionals;
"""
query_gpspoint = """
select course_cd, race_date, race_number, saddle_cloth_number, time_stamp, longitude, latitude,
    speed, progress, stride_frequency
from v_gpspoint;
"""

In [9]:
# Execute the query and load data into a DataFrame
gps_df = pd.read_sql_query(query_gpspoint, engine, parse_dates=['time_stamp'])
sectionals_df = pd.read_sql_query(query_sectionals, engine)
results_df = pd.read_sql_query(query_results, engine)

In [10]:
gps_df['race_date'] = pd.to_datetime(gps_df['race_date'])
sectionals_df['race_date'] = pd.to_datetime(sectionals_df['race_date'])
results_df['race_date'] = pd.to_datetime(results_df['race_date'])

In [11]:
gps_df.to_parquet('/home/exx/myCode/horse-racing/FoxRiverAIRacing/notebooks/data/gps.parquet', index=False)
sectionals_df.to_parquet('/home/exx/myCode/horse-racing/FoxRiverAIRacing/notebooks/data/sectionals.parquet', index=False)
results_df.to_parquet('/home/exx/myCode/horse-racing/FoxRiverAIRacing/notebooks/data/results.parquet', index=False)


# Start here if using same frames

In [16]:
import pandas as pd

# Load the Parquet file into a DataFrame
gps_df = pd.read_parquet('/home/exx/myCode/horse-racing/FoxRiverAIRacing/notebooks/data/gps.parquet')
sectionals_df = pd.read_parquet('/home/exx/myCode/horse-racing/FoxRiverAIRacing/notebooks/data/sectionals.parquet')
results_df = pd.read_parquet('/home/exx/myCode/horse-racing/FoxRiverAIRacing/notebooks/data/results.parquet')

# Display the DataFrames
# print(gps_df.head())
# print(sectionals_df.head())
# print(results_df.head())

In [17]:
print(gps_df.shape)
print(results_df.shape)
print(sectionals_df.shape)

(48014372, 10)
(388056, 6)
(5603785, 11)


In [18]:
# Join gps_df with results_df
merged_df = gps_df.merge(results_df, 
                          left_on=['course_cd', 'race_date', 'race_number', 'saddle_cloth_number'], 
                          right_on=['course_cd', 'race_date', 'race_number', 'program_num'], 
                          how='inner')

In [19]:
print(merged_df.shape)
print(sectionals_df.shape)

(34194909, 13)
(5603785, 11)


In [20]:
# Join the result with sectionals_df
merged_df = merged_df.merge(sectionals_df, 
                             left_on=['course_cd', 'race_date', 'race_number', 'saddle_cloth_number'], 
                             right_on=['course_cd', 'race_date', 'race_number', 'saddle_cloth_number'], 
                             how='inner')


In [21]:
print(merged_df.shape)

(488547778, 20)


In [22]:
print(merged_df.dtypes)

course_cd                      object
race_date              datetime64[ns]
race_number                     int64
saddle_cloth_number            object
time_stamp             datetime64[ns]
longitude                     float64
latitude                      float64
speed                         float64
progress                      float64
stride_frequency              float64
program_num                    object
official_fin                    int64
post_pos                        int64
gate_numeric                  float64
length_to_finish              float64
sectional_time                float64
running_time                  float64
distance_back                 float64
distance_ran                  float64
number_of_strides             float64
dtype: object


In [23]:
# Check for missing values
print(merged_df.isnull().sum())

course_cd                     0
race_date                     0
race_number                   0
saddle_cloth_number           0
time_stamp                    0
longitude                     0
latitude                      0
speed                         0
progress                      0
stride_frequency       41832491
program_num                   0
official_fin                  0
post_pos                      0
gate_numeric                  0
length_to_finish              0
sectional_time                0
running_time                  0
distance_back              5480
distance_ran                  0
number_of_strides        242196
dtype: int64


In [ ]:
import pandas as pd

# Define the chunk size
chunk_size = 100000  # Adjust this size based on your memory capacity

# Create a copy of the DataFrame to avoid modifying the original during iteration
merged_df_copy = merged_df.copy()

# Iterate through the DataFrame in chunks
for start in range(0, len(merged_df_copy), chunk_size):
    end = start + chunk_size
    chunk = merged_df_copy.iloc[start:end]
    
    # Impute missing stride_frequency based on official_fin
    for index, row in chunk.iterrows():
        if pd.isnull(row['stride_frequency']):
            avg_stride_freq = merged_df_copy[merged_df_copy['official_fin'] == row['official_fin']]['stride_frequency'].mean()
            merged_df_copy.at[index, 'stride_frequency'] = avg_stride_freq

# Replace the original DataFrame with the updated one
merged_df['stride_frequency'] = merged_df_copy['stride_frequency']

In [ ]:
print(merged_df.dtypes)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate correlation matrix
corr_matrix = df[['speed', 'progress', 'stride_frequency', 'longitude', 'latitude', 'post_pos', 'official_fin']].corr()

# Plot heatmap
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Define your variables
max_seq_length = 120  # Replace with your maximum sequence length
num_features = 5      # Replace with the actual number of features in your data
num_classes = 12      # Replace with the actual number of classes

# Build your model
model_lstm = tf.keras.Sequential()
model_lstm.add(tf.keras.layers.Masking(mask_value=0., input_shape=(max_seq_length, num_features)))
model_lstm.add(tf.keras.layers.LSTM(128))
model_lstm.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model_lstm.summary()

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
# Load data into dataframe:

import pandas as pd

In [ ]:
# Training

history_lstm = model_lstm.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# Combining the Models

To create an ensemble, you can combine the predictions of these models in several ways:
	1.	Averaging Probabilities:
	•	Obtain probability distributions over finishing positions from each model.
	•	Average the probabilities across models to get the final prediction.
	2.	Weighted Averaging:
	•	Assign weights to each model based on validation performance.
	•	Compute a weighted average of the probabilities.
	3.	Stacking (Meta-Learner):
	•	Use the predictions from the individual models as input features to a meta-model (e.g., a logistic regression or another neural network).
	•	The meta-model learns how to best combine the individual predictions.
	4.	Voting (for Classification):
	•	If treating the problem as classification into discrete positions, use majority voting among the models.
	•	Not as suitable if you need probability distributions.

Implementation Steps

1. Data Preparation

	•	Sequences:
	•	Use the raw GPS data (gpspoint) to create sequences for each horse in each race.
	•	Ensure that sequences are properly sorted by time_stamp.
	•	Features:
	•	Include raw features such as speed, progress, stride_frequency.
	•	Avoid hand-engineering features like acceleration to adhere to your objective.
	•	Labels:
	•	Use official_fin from results_entries as the target variable.
	•	Since you want probabilities for each finishing position, consider encoding official_fin as categorical labels.
